In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time # Thư viện để tạm dừng

In [33]:
# --- 1. Khai báo thông tin ---
URL = "https://homedy.com/ban-nha-dat"
# Giả lập là một trình duyệt để tránh bị chặn
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- 2. Gửi yêu cầu và lấy nội dung HTML ---

In [34]:
# --- 2. Gửi yêu cầu và lấy nội dung HTML ---
try:
    response = requests.get(URL, headers=HEADERS)
    response.raise_for_status() # Báo lỗi nếu request không thành công
except requests.exceptions.RequestException as e:
    print(f"Lỗi khi gửi yêu cầu: {e}")
    exit()

In [35]:
response.content

b'<!DOCTYPE html>\r\n<html lang="vi" prefix="og: http://ogp.me/ns#">\r\n<head>\r\n    <title>Mua B\xc3\xa1n Nh\xc3\xa0 \xc4\x90\xe1\xba\xa5t T\xe1\xba\xa1i Vi\xe1\xbb\x87t Nam Gi\xc3\xa1 R\xe1\xba\xbb, V\xe1\xbb\x8b Tr\xc3\xad \xc4\x90\xe1\xba\xb9p (10/2025) </title>\r\n    <meta name="description" content="Mua b\xc3\xa1n nh\xc3\xa0 \xc4\x91\xe1\xba\xa5t t\xe1\xba\xa1i Vi\xe1\xbb\x87t Nam gi\xc3\xa1 r\xe1\xba\xbb 2023 \xe2\x9c\x85 B\xc3\xa1o gi\xc3\xa1 mua b\xc3\xa1n nh\xc3\xa0 \xc4\x91\xe1\xba\xa5t Vi\xe1\xbb\x87t Nam s\xe1\xbb\x95 \xc4\x91\xe1\xbb\x8f ch\xc3\xadnh ch\xe1\xbb\xa7: Nh\xc3\xa0 ri\xc3\xaang, chung c\xc6\xb0, nh\xc3\xa0 ph\xe1\xbb\x91, bi\xe1\xbb\x87t th\xe1\xbb\xb1 li\xe1\xbb\x81n k\xe1\xbb\x81, \xc4\x91\xe1\xba\xa5t n\xe1\xbb\x81n..." />\r\n    <meta name="keywords" content="mua b\xc3\xa1n nh\xc3\xa0 \xc4\x91\xe1\xba\xa5t, mua ban nha dat, nh\xc3\xa0 \xc4\x91\xe1\xba\xa5t, nha dat, b\xc3\xa1n nh\xc3\xa0 \xc4\x91\xe1\xba\xa5t, ban nha dat" />\r\n    <meta charset="utf-8"

In [36]:
# --- 3. Phân tích HTML ---
soup = BeautifulSoup(response.content, 'html.parser')

In [37]:
# --- 4. Tìm tất cả các tin đăng ---
# *** BẠN PHẢI THAY THẾ 'class_cua_the_cha_chua_tin_dang' ***
# Ví dụ: soup.find_all('div', class_='re__product-item')
list_items = soup.find_all('div', class_='product-content')
print(list_items)

[<div class="product-content">
<h3><a class="title padding-hoz" href="/ban-can-ho-vinhomes-ocean-park-3-the-crown/ra-mat-cum-chung-cu-masterise-cuoi-cung-chuan-bi-mo-es3173840" rel="follow" title="Ra mắt Cụm Chung cư Masterise cuối cùng Vinhomes Ocean Park 3 chuẩn bị mở bán ">Ra mắt Cụm Chung cư Masterise cuối cùng Vinhomes Ocean Park 3 chuẩn bị mở bán </a></h3>
<div class="description padding-hoz">GIÁ RUMOR CHUNG CƯ VINHOMES OCEAN PARK CỰC HOT!!!+ Chỉ từ 6x - 7x chỉ từ 6x triệu/m²  cạnh tranh nhất phân khúc cao cấp Masterise+ 2 tầng hầm, Mỗi căn...</div>
<ul class="product-unit">
<li><i class="icon icon-price"></i> <span class="price">1,9 - 6,5 Tỷ</span></li>
<li class="acr"><i class="icon icon-acr"></i> <span class="acreage">28 - 112 m2</span></li>
<li>58 tr/m2</li>
<li class="address" title="Tân Quang, Thị trấn Như Quỳnh, Văn Lâm và Nghĩa Trụ, Văn Giang, tỉnh Hưng Yên"><i class="icon icon-address"></i>  Văn Giang,  tỉnh Hưng Yên</li>
</ul>
<div class="box-price-agency">
<div class="

In [38]:
data = []
for item in list_items:
    try:
        # Trích xuất giá
        price_element = item.find('span', class_='price')
        price = price_element.get_text(strip=True) if price_element else "Không rõ"
        
        # Trích xuất diện tích
        acreage_element = item.find('span', class_='acreage')
        acreage = acreage_element.get_text(strip=True) if acreage_element else "Không rõ"
        
        # Trích xuất địa chỉ
        address_element = item.find('li', class_='address')
        address = address_element.get_text(strip=True) if address_element else "Không rõ"
        
        data.append({
            'Địa chỉ': address,
            'Diện tích': acreage,
            'Giá': price
        })
    except Exception as e:
        print(f"Lỗi khi trích xuất dữ liệu: {e}")
        continue

In [39]:
if data:
    df = pd.DataFrame(data)
    print(df)
    df.to_csv('vietnam_housing_data.csv', index=False, encoding='utf-8-sig')
    print("Đã lưu dữ liệu vào file 'vietnam_housing_data.csv'")
else:
    print("Không tìm thấy dữ liệu nào để lưu!")

                               Địa chỉ    Diện tích               Giá
0            Văn Giang,  tỉnh Hưng Yên  28 - 112 m2      1,9 - 6,5 Tỷ
1     Quận Bình Thạnh,  TP Hồ Chí Minh        50 m2              2 Tỷ
2             Quận 10,  TP Hồ Chí Minh        63 m2         970 Triệu
3   quận Ninh Kiều,  thành phố Cần Thơ        70 m2            1,5 Tỷ
4       Thành phố Quy Nhơn,  Bình Định   50 - 96 m2      2,4 - 7,9 Tỷ
5       Thành phố Quy Nhơn,  Bình Định  70 - 150 m2       5,2 - 10 Tỷ
6       Thành phố Quy Nhơn,  Bình Định   35 - 58 m2  499 - 1024 Triệu
7    Huyện Bình Chánh,  TP Hồ Chí Minh        97 m2            1,2 Tỷ
8              Quận 8,  TP Hồ Chí Minh      59,5 m2           1,05 Tỷ
9            Quận Nam Từ Liêm,  Hà Nội        80 m2           22,5 Tỷ
10    Quận Bình Thạnh,  TP Hồ Chí Minh        65 m2         950 Triệu
11             Quận 7,  TP Hồ Chí Minh        60 m2           1,69 Tỷ
12   Huyện Bình Chánh,  TP Hồ Chí Minh        85 m2         970 Triệu
13        Quận Gò Vấ